In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import zipfile

# WRIME データセットのロード
dataset = load_dataset("izumi-lab/wrime")

# 東北大学の事前学習済みモデルをロード
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8)

# データの前処理
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 訓練パラメータ設定
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# モデルの訓練
trainer.train()

# 学習済みモデルの保存
model_path = "trained_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# モデルをZIP化してダウンロード用に準備
zip_filename = "trained_model.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.write(model_path + "/config.json", "trained_model/config.json")
    zf.write(model_path + "/pytorch_model.bin", "trained_model/pytorch_model.bin")
    zf.write(model_path + "/tokenizer.json", "trained_model/tokenizer.json")
    zf.write(model_path + "/vocab.txt", "trained_model/vocab.txt")

print("Download trained_model.zip from the Colab file system")
